In [1]:
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 40.7 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.4 MB/s eta 0:00:0000:0100:01
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.3 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 81.3 MB/s eta 0:00:00
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 87.9 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 89.0 MB/s eta 0:00:00ta 0:

Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, Input, Reshape, Flatten
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
import os
from sklearn.svm import SVC
import scipy.linalg as la
from numpy.random import seed
import gurobipy as gp
from gurobipy import GRB
seed(1)
tensorflow.random.set_seed(2)

2022-10-08 00:20:25.724917: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-08 00:20:25.848958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-08 00:20:25.848994: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-08 00:20:25.875300: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-08 00:20:26.477053: W tensorflow/stream_executor/pla

In [5]:
#optimization problem to get the high dimensional data point from low dimensional data point
#datapoint_low -> low dimensional data point
#G -> projection matrix
#lb -> lower bounds of the features
#ub -> upper bounds of the features
#C,epsilon -> constants in the optimization problem
#N -> high dimensional feature size
#zi_type -> vector of size N representing whether a feature is integer value(1) or real value(0)
def GetHighDimensionDataset(datapoint_low, G, lb, ub, C, epsilon, N, zi_type):
    datapoint_low = datapoint_low.reshape(datapoint_low.shape[0],)
    lb = lb.reshape(lb.shape[0])
    ub = ub.reshape(ub.shape[0])
    K = datapoint_low.shape[0]
    eps_vector = epsilon * np.ones((K,))
    zi_type = zi_type.reshape(-1)

    attr_index = np.arange(zi_type.shape[0])
    zi_int_index = attr_index[zi_type==1]
    zi_real_index = attr_index[zi_type==0]
    zi_int_lb = lb[zi_int_index]
    zi_int_ub = ub[zi_int_index]
    zi_real_lb = lb[zi_real_index]
    zi_real_ub = ub[zi_real_index]
    G_int = G[zi_int_index,:]
    G_real = G[zi_real_index,:]

    #using gurobipy   
    if zi_int_index.shape[0]==0 and zi_real_index.shape[0]!=0: #only real attr
      model = gp.Model("eigenbag")
      model.Params.IterationLimit = iter_lim #GRB.INFINITY
      zi_real = model.addMVar ( zi_real_index.shape[0], vtype=GRB.CONTINUOUS, name="zi_real" )
      qip = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qip" )
      qim = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qim" )
      model.addConstr( (G_real.T)@zi_real - qip <= datapoint_low + eps_vector, name="c1")
      model.addConstr( -(G_real.T)@zi_real - qim <= -datapoint_low + eps_vector, name="c2")
      model.addConstr( zi_real <= zi_real_ub , name="c3")
      model.addConstr( zi_real >= zi_real_lb , name="c4")

      model.setObjective(( C*qip.sum() + C*qim.sum() ), sense=GRB.MINIMIZE)
      # Verify model formulation
      model.write('eigenbag.lp')
      # Run optimization engine
      model.optimize()
      try:
        zi_real = (zi_real.x)
        zi_int = np.array([])
      except:
        return np.random.rand(N, 1),1
  
    elif zi_int_index.shape[0]!=0 and zi_real_index.shape[0]==0: #only int attr
      model = gp.Model("eigenbag")
      model.Params.IterationLimit = iter_lim #GRB.INFINITY
      zi_int = model.addMVar ( zi_int_index.shape[0], vtype=GRB.INTEGER, name="zi_int" )
      qip = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qip" )
      qim = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qim" )
      model.addConstr( (G_int.T)@zi_int - qip <= datapoint_low + eps_vector, name="c1")
      model.addConstr( -(G_int.T)@zi_int - qim <= -datapoint_low + eps_vector, name="c2")
      model.addConstr( zi_int <= zi_int_ub , name="c3")
      model.addConstr( zi_int >= zi_int_lb , name="c4")

      model.setObjective(( C*qip.sum() + C*qim.sum() ), sense=GRB.MINIMIZE)
      # Verify model formulation
      model.write('eigenbag.lp')
      # Run optimization engine
      model.optimize()
      try:
        zi_int = (zi_int.x)
        zi_real = np.array([])
      except:
        return np.random.rand(N, 1),1

    else: #both real and int attr
      model = gp.Model("eigenbag")
      model.Params.IterationLimit = iter_lim#1000000.0#GRB.INFINITY
      zi_real = model.addMVar ( zi_real_index.shape[0], vtype=GRB.CONTINUOUS, name="zi_real" )
      zi_int = model.addMVar ( zi_int_index.shape[0], vtype=GRB.INTEGER, name="zi_int" )
      qip = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qip" )
      qim = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qim" )
      model.addConstr( (G_int.T)@zi_int + (G_real.T)@zi_real - qip <= datapoint_low + eps_vector, name="c1")
      model.addConstr( -(G_int.T)@zi_int -(G_real.T)@zi_real - qim <= -datapoint_low + eps_vector, name="c2")
      model.addConstr( zi_int <= zi_int_ub , name="c3")
      model.addConstr( zi_int >= zi_int_lb , name="c4")
      model.addConstr( zi_real <= zi_real_ub , name="c5")
      model.addConstr( zi_real >= zi_real_lb , name="c6")

      model.setObjective(( C*qip.sum() + C*qim.sum() ), sense=GRB.MINIMIZE)
      # Verify model formulation
      model.write('eigenbag.lp')
      # Run optimization engine
      model.optimize()
      try:
        zi_int = (zi_int.x)
        zi_real = (zi_real.x)
      except:
        return np.random.rand(N, 1),1

    #rounding the values to nearest integers for integer attributes and making sure the projected back attributes lie between the lower and upper bounds
    zi_int_rounded = np.empty(shape=[0])
    zi_real_rounded = np.empty(shape=[0])
    for i in range(zi_int.shape[0]):
      st_list = [st for st in range(int(zi_int_lb[i]),int(zi_int_ub[i]+1),1)]
      dist = [abs(zi_int[i]-st) for st in range(int(zi_int_lb[i]),int(zi_int_ub[i]+1),1)]
      minpos = dist.index(min(dist))
      rz = np.array([st_list[minpos]])
      zi_int_rounded = np.append(zi_int_rounded,rz,axis=0)
    for i in range(zi_real.shape[0]):
      if zi_real[i] >= zi_real_ub[i]:
        rz = zi_real_ub[i]
      elif zi_real[i] <= zi_real_lb[i]:
        rz = zi_real_lb[i]
      else:
        rz = zi_real[i]
      zi_real_rounded = np.append(zi_real_rounded,np.array([rz]),axis=0)

    zi_unsorted = np.hstack((zi_int_rounded,zi_real_rounded))
    zi_index_unsorted = np.hstack((zi_int_index,zi_real_index))
    zi =  np.empty(shape=[0])
    for i in range(zi_unsorted.shape[0]):
      zi = np.append(zi,zi_unsorted[zi_index_unsorted==i],axis=0)

    datapoint_high = zi
    return datapoint_high,0

In [6]:
#V -> eigenvectors
#D -> eigenvalues
def SortEigenvalues(V, D):
    # sort matrix of eigenvalues
    idx = D.argsort()[::-1]
    # obtain indices of sorted eigenvalues
    D_sort = D[idx]
    # arrange the eigenvectors in the order of sorted eigenvalues
    V_sort = V[:,idx]
    return V_sort, D_sort

In [7]:
#input_dataset -> projected dataset belonging to the cluster
#cluster_center -> cluster center
def FindMidPointDistancesFromClusterCenters(input_dataset, cluster_center):
    # Compute size of dataset
    N, D = input_dataset.shape[0],input_dataset.shape[1]
    # Initialize output dataset matrix
    output_dataset = np.empty(shape = (N,D))
    # Iterate over samples
    for i in range(N):
        # compute the sample as the mid point of the line joining the sample with the corresponding cluster center
        output_dataset[i,:] = np.add(input_dataset[i,:],cluster_center)/2
    return output_dataset

In [8]:
#EVs_vec -> eigenvectors
#eigvar -> required minimum variance
def FindBestEVs(EVs_vec,eigvar): 
    # Compute ratio of variance preserved by eigenvalues
    if eigvar==1:
      sel_EVs = len(EVs_vec)
      return sel_EVs
    else:
      for i in range(len(EVs_vec)):
        sum_ratio = sum([x**2 for x in EVs_vec[0:i+1]]) / sum([x**2 for x in EVs_vec])
        # check if variance captured is > eigvar
        if (sum_ratio >= eigvar):
          # select i eigenvectors
          sel_EVs = i+1
          return sel_EVs

In [9]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from numpy import cov
import collections
#features,labels -> X,y
#Cparam,epsilon -> constants for optimization problem
#zi_type -> vector of size N representing whether a feature is integer value(1) or real value(0)
#eigvar -> required minimum variance
def AugmentDataset(features, labels, Cparam, epsilon, zi_type, eigvar):
    # Compute dataset size
    N, dim = features.shape[0],features.shape[1]
    
    zi_lb = np.min(features,axis=0)
    zi_ub = np.max(features,axis=0)

    corelmatrix = np.corrcoef(features.T)
    corelmatrix[np.isnan(corelmatrix)] = 0    # Sanity check!!
    # Compute eigenvalues and eigenvectors of correlation matrix
    D, V = la.eig(corelmatrix)
    # Sort the eigenvectors in order of decreasing eigenvalues
    V1, D1 = SortEigenvalues(V, D)
    # Choose the k largest eigenvalues and their corresponding eigenvectors
    k = FindBestEVs(D1,eigvar)
    V_selected = V1[:, 0:k]
    # Compute projections of the samples onto the k eigendirections
    projected_dataset = features @ V_selected
    projected_dataset = np.real(projected_dataset)
    # Choose number of clusters by rule of thumb
    k_clusters = math.floor(math.sqrt(N / 2))
    # Perform k-means clustering
    kms=KMeans(n_clusters=k_clusters)
    kms.fit(projected_dataset)
    IDX = kms.predict(projected_dataset)
    C =kms.cluster_centers_
    # construct lines joining the cluster centres with each point of the
    # respective cluster, and then compute the mid-points of these lines
    new_dataset = np.array([])
    new_labels = np.array([])
    for i in range(k_clusters):
        new_dataset_for_cluster = FindMidPointDistancesFromClusterCenters(projected_dataset[IDX == i,:], C[i, :])
        if new_dataset.shape[0]==0:
            new_dataset = new_dataset_for_cluster
        else:
            new_dataset = np.vstack((new_dataset, new_dataset_for_cluster ))

    new_dataset_high_dimension = np.empty(shape=(new_dataset.shape[0],dim))
    for i in range(new_dataset.shape[0]):
        newhd,flag = GetHighDimensionDataset(new_dataset[i, :], V_selected, zi_lb, zi_ub, Cparam, epsilon, dim, zi_type)
        if flag==0:
          new_dataset_high_dimension[i, :] = newhd
    
    #finding the class of the generated samples
    if len(np.unique(labels.reshape(-1)))>1:
      sc = StandardScaler().fit(features)
      features_sc = sc.transform(features)
      new_dataset_high_dimension_sc = sc.transform(new_dataset_high_dimension)
      model = RandomForestClassifier(n_estimators=1000, class_weight='balanced')
      model.fit(features_sc, labels)
      new_labels = (model.predict(new_dataset_high_dimension_sc)).reshape(-1,1)
    else:
      print('lab classes:'+str(np.unique(labels.reshape(-1))))
      new_labels = labels[0][0]*np.ones((new_dataset.shape[0],1))
    
    return new_dataset_high_dimension, new_labels

In [10]:
import random
def random_combination(iterable,r,number_of_comb):
    i = 0
    pool = tuple(iterable)
    n = len(pool)
    rng = range(n)
    while i < number_of_comb:
        i += 1
        yield [pool[j] for j in random.sample(rng, r)] 

In [11]:
def find_similarity(A,B):
  sim_mat = cosine_similarity(A, B)
  sim_add = 0
  for i in range(sim_mat.shape[0]):
      sim_add = sim_add + abs(sim_mat[i][i])
  return sim_add/sim_mat.shape[0]

In [12]:
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy import spatial
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics.pairwise import cosine_similarity

def generate_samples(x_train,y_train,number_of_classes,Cparam,epsilon,samp_t,zi_type,eigvar):
  genDataTrainXFinal = np.empty(shape=[0, x_train.shape[1]])
  genDataTrainYFinal = np.empty(shape=[0,1])
  out_thres = 0.9 
  gen_mul_factor = 1
  gen_mul_factor_max = 10 #maximum number of times generated samples are computed
  in_thres = 0.98
  inner_counter = 0
  outer_counter = 0
  sim_outer_list = list()
  sim_inner_list = list() 
  while gen_mul_factor <= gen_mul_factor_max:
    print('outer_counter:'+str(outer_counter))
    print('inner_counter:'+str(inner_counter))
    if outer_counter%10==0 and outer_counter!=0:
      out_thres = np.mean(np.asarray(sim_outer_list))
      print('new out_thres:'+str(out_thres))
    if inner_counter%5==0 and inner_counter!=0:
      in_thres = np.mean(np.asarray(sim_inner_list))
      print('new in_thres:'+str(in_thres))
    for j in random_combination(range(x_train.shape[0]), samp_t, 1):
      data_train_used=x_train[j,:]
      labels_train_used=y_train[j,:]
      pca = PCA(n_components = 0.98)
      pca.fit(x_train)
      feature_weights_x_train = pca.components_
      n_components_for_x_train = feature_weights_x_train.shape[0]
      pca = PCA(n_components = n_components_for_x_train)
      pca.fit(data_train_used)
      feature_weights_data_train_used = pca.components_
      #finding similarity between the pca components of original dataset and dataset that will be used to generate samples
      sim = find_similarity(feature_weights_x_train,feature_weights_data_train_used)
      print('similarity:'+str(sim))
      sim_outer_list.append(abs(sim))
      outer_counter += 1
      if abs(sim)> out_thres:
        genDataX,genDataY = AugmentDataset(data_train_used,labels_train_used,Cparam,epsilon,zi_type,eigvar) 
        pca = PCA(n_components = 0.98)
        pca.fit(data_train_used)
        feature_weights_data_train_used = pca.components_
        n_components_for_data_train_used = feature_weights_data_train_used.shape[0]
        pca = PCA(n_components = n_components_for_data_train_used)
        pca.fit(genDataX)
        feature_weights_gen = pca.components_
        #finding similarity between the pca components of dataset being used to generate samples and the generated samples
        sim = find_similarity(feature_weights_data_train_used,feature_weights_gen)
        print('similarity.........:'+str(sim))
        sim_inner_list.append(abs(sim))
        inner_counter += 1
        if abs(sim)>in_thres:
          genDataTrainXFinal=np.append(genDataTrainXFinal,genDataX,axis=0)
          genDataTrainYFinal=np.append(genDataTrainYFinal,genDataY,axis=0)
          print('Added')
          print('generated_data_size:'+str(genDataTrainXFinal.shape[0]))
          gen_mul_factor+=1
        else:
          print('Discarded')
          print('generated_data_size:'+str(genDataTrainXFinal.shape[0]))
  return genDataTrainXFinal,genDataTrainYFinal

In [13]:
import tempfile
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
def train_test(orig_X,orig_Y,generated_X,generated_Y,\
               total_X,total_Y,test_X,test_Y):
  sc = StandardScaler().fit(orig_X)
  orig_X_sc = sc.transform(orig_X)
  test_X_1_sc = sc.transform(test_X)

  sc = StandardScaler().fit(total_X)
  total_X_sc = sc.transform(total_X)
  test_X_3_sc = sc.transform(test_X)

  no_of_classes = len(np.unique(y.reshape(-1)))
  encoded_orig_Y = orig_Y
  dummy_orig_Y = np_utils.to_categorical(encoded_orig_Y,num_classes=no_of_classes)
  encoded_total_Y = total_Y
  dummy_total_Y = np_utils.to_categorical(encoded_total_Y,num_classes=no_of_classes)
  encoded_test_Y = test_Y
  dummy_test_Y = np_utils.to_categorical(encoded_test_Y,num_classes=no_of_classes)

  model1 = Sequential()
  model1.add(Dense(1.5*orig_X.shape[1], input_dim=orig_X.shape[1], activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
  model1.add(Dense(orig_X.shape[1], activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
  model1.add(Dense(no_of_classes, activation='softmax',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
	# print(model1.summary())
  # Compile model
  model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  es1 = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)
  mc1 = ModelCheckpoint('best_model1.h5', monitor='val_accuracy', mode='max', save_best_only=True)
  history=model1.fit(orig_X_sc,dummy_orig_Y,validation_data=(test_X_1_sc, dummy_test_Y),epochs=50,batch_size=min(int(0.2*orig_X.shape[0]),100),callbacks=[es1, mc1])
  saved_model1 = load_model('best_model1.h5')
  _, accuracy_orig = saved_model1.evaluate(test_X_1_sc, dummy_test_Y)
  print('Accuracy_orig: %.2f' % (accuracy_orig*100))
    
  model3 = Sequential()
  model3.add(Dense(1.5*orig_X.shape[1], input_dim=orig_X.shape[1], activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
  model3.add(Dense(orig_X.shape[1], activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
  model3.add(Dense(no_of_classes, activation='softmax',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
	# print(model3.summary())
  # Compile model
  model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  es3 = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)
  mc3 = ModelCheckpoint('best_model3.h5', monitor='val_accuracy', mode='max', save_best_only=True)
  history=model3.fit(total_X_sc,dummy_total_Y,validation_data=(test_X_3_sc, dummy_test_Y),epochs=50,batch_size=min(int(0.2*total_X.shape[0]),100),callbacks=[es3, mc3])
  saved_model3 = load_model('best_model3.h5')
  _, accuracy_tot = saved_model3.evaluate(test_X_3_sc, dummy_test_Y)
  print('Accuracy_tot: %.2f' % (accuracy_tot*100))

  return accuracy_orig,accuracy_tot

Parameters that can be changed:
1. Cparam
2. epsilon
3. threshold value(outer and inner)
4. gen_mul_factor
5. samples at a time
6. eigenvar


In [ ]:
from scipy.io import loadmat
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import math
from sklearn.model_selection import train_test_split
#Required files:
#*.mat -> containing features(X) and classes(y)
#*fold.mat -> containing the indices of each fold
#*info.mat -> containg an array of the feature size denoting whether a feature is int valued(1) or real valued(0) 
# Define parameters for EigenSample - these can be changed according to the dataset
Cparam = 1
epsilon = 0.01
dataset_list=[1]#[13,16,19,2,21,23,24,26,27,5]
iter_lim = 1e2 #GRB.INFINITY

for dataset in dataset_list:
  Results = list()
  Results_arr = list()
  print('dataset:'+str(dataset))
  #annots = loadmat('{}.mat'.format(dataset))
  #annotsi = loadmat('{}fold.mat'.format(dataset))
  
  #annotsinf = loadmat('{}info.mat'.format(dataset))

  #X = annots['x']
  #y = annots['y']
  X = np.random.randn(100,5) #TODO reshape(-1,1) if 1D
  y = np.random.randint(0,1,(100,1)) #TODO reshape(-1,1) if 1D
  #print(X.shape)
  #print(y)
  #indices = annotsi['indices']
  #print(indices.shape)
  
  #indices = indices.reshape(indices.shape[0])
  #print(indices)

  indices = np.random.randint(1,4,(100)) #TODO just randonmly make it with batch index values (till maybe 3/4); same size as y
  #indices = indc
  #zi_type = annotsinf['zi_type']
  zi_type = np.zeros([X.shape[1]])
  y = y.reshape(y.shape[0])
  for i in range(y.shape[0]):
    if y[i] == -1:
      y[i] = 0  

  y_fold_min = math.inf
  for i in np.unique(indices):
    y_fold=y[indices!=i]
    y_fold_min = min(y_fold_min,y_fold.shape[0])

  for samples_at_a_time in [y_fold_min-2]:#TODO [y_fold_min-2,y_fold_min-5] keep it in this range (or even y_fold_min)
    for eigenvariance in [1]:#[1,0.98,0.95,0.9]:
      acc_orig_list = list()
      acc_tot_list = list()
      x_tr_list = list()
      tot_x_list = list() 
      y = y.reshape(-1,1)
      count = 1
      for i in np.unique(indices):
        X_Te = X[indices==i]
        y_Te = y[indices==i]
        X_Tr = X[indices!=i]
        y_Tr = y[indices!=i]

        gen_X,gen_y= generate_samples(X_Tr,y_Tr,len(np.unique(y_Tr.reshape(-1))),Cparam, epsilon, \
                                      samples_at_a_time,zi_type,eigenvariance)
        
        print("gen_X_shape:"+str(gen_X.shape))
        print("gen_y_shape:"+str(gen_y.shape))
        tot_X=np.append(X_Tr,gen_X,axis=0)
        tot_y=np.append(y_Tr,gen_y,axis=0)
        
        if count == 1:
            X_final = tot_X
            Y_final = tot_y
        else:
            X_final = np.append(X_final, tot_X, axis = 0)
            Y_final = np.append(Y_final, tot_y, axis = 0)
        count = count+1
        
        print("tot_X_shape:"+str(tot_X.shape))
        print("tot_y_shape:"+str(tot_y.shape))
        x_tr_list.append(X_Tr.shape[0])
        tot_x_list.append(tot_X.shape[0])
        try:
          acc_orig,acc_tot=train_test(X_Tr,y_Tr,gen_X,gen_y,tot_X,tot_y,X_Te,y_Te)
          acc_orig_list.append(acc_orig)
          acc_tot_list.append(acc_tot)
        except:
          print('except_occured')
      
      acc_orig_arr = np.array(acc_orig_list)
      acc_tot_arr = np.array(acc_tot_list)
      acc_orig_mean = np.mean(acc_orig_arr)
      acc_tot_mean = np.mean(acc_tot_arr)
      acc_orig_std = np.std(acc_orig_arr)
      acc_tot_std = np.std(acc_tot_arr)
      
      dic = dict()
      li = list()
      dic['dataset'] = dataset
      li.append(dataset)
      dic['parameter'] = samples_at_a_time
      li.append(samples_at_a_time)
      dic['eigenvar'] = eigenvariance
      li.append(eigenvariance)
      dic['orig_shape_0'] = np.mean(np.array(x_tr_list))
      li.append(X_Tr.shape[0])
      dic['orig_shape_1'] = X_Tr.shape[1]
      li.append(X_Tr.shape[1])
      dic['acc_orig_mean'] = acc_orig_mean
      li.append(acc_orig_mean)
      dic['acc_orig_std'] = acc_orig_std
      li.append(acc_orig_std)
      dic['orig_acc_fold'] = str(acc_orig_list).replace(',','/') 
      li.append(str(acc_orig_list).replace(',','/') )
      dic['tot_shape_0'] = np.mean(np.array(tot_x_list))
      li.append(tot_X.shape[0])
      dic['acc_tot_mean'] = acc_tot_mean
      li.append(acc_tot_mean)
      dic['acc_tot_std'] = acc_tot_std
      li.append(acc_tot_std)
      dic['tot_acc_fold'] = str(acc_tot_list).replace(',','/')  
      li.append(str(acc_tot_list).replace(',','/') )
      #np.savetxt("eigenbag_dataset_{}_samples_{}_eigenvar_{}.csv".format(dataset,samples_at_a_time,eigenvariance),np.array(li).reshape(1,len(li)),fmt= '%s',delimiter=',',\
      #       header='dataset, parameter, eigenvar, orig_shape_0, orig_shape_1, acc_orig_mean, acc_orig_std, orig_acc_fold, tot_shape_0, acc_to_mean, acc_tot_std, tot_acc_fold')
      Results.append(dic) 
      Results_arr.append(li)
  print("Results:")
  print(Results)
  Results_arr=np.array(Results_arr)
  np.savetxt("Results_eigenbag_{}.csv".format(dataset),Results_arr,fmt= '%s',delimiter=',',\
             header='dataset, parameter, eigenvar, orig_shape_0, orig_shape_1, acc_orig_mean, acc_orig_std, orig_acc_fold, tot_shape_0, acc_to_mean, acc_tot_std, tot_acc_fold')
             

dataset:1
outer_counter:0
inner_counter:0
similarity:0.7685793909289702
outer_counter:1
inner_counter:0
similarity:0.8420641884443387
outer_counter:2
inner_counter:0
similarity:0.8345720147815859
outer_counter:3
inner_counter:0
similarity:0.6492587558873245
outer_counter:4
inner_counter:0
similarity:0.6222610519973972
outer_counter:5
inner_counter:0
similarity:0.6880558352547614
outer_counter:6
inner_counter:0
similarity:0.9761704096372587
Restricted license - for non-production use only - expires 2023-10-25
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0x27a81c07
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-02, 3e+00]
Presolve removed 10 rows and 0 columns
Presolve time: 0.01s


  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 3e+00]
Presolve removed 12 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.5934025e-01   2.948245e+00   0.000000e+00      0s
       5    2.0725492e+00   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.072549190e+00
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0x3391b7cc
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-02, 3e+00]
Presolve removed 12 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 48 nonzeros

Iteration   

  RHS range        [5e-01, 3e+00]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.016857e+00   0.000000e+00      0s
       9    1.1970444e-01   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.197044443e-01
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0x563b5243
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-02, 3e+00]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    

Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.398829e+00   0.000000e+00      0s
       7    1.2736426e-01   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.273642644e-01
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0x8d79d73d
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 3e+00]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.

Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.7628406e-01   6.479003e+00   0.000000e+00      0s
       7    2.4307399e+00   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.430739920e+00
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0xafd3686d
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 3e+00]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.592102e+00   0.000000e+0

...

Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.759806e+00   0.000000e+00      0s
       8    1.4944396e+00   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.494439583e+00
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0x98e18a23
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 3e+00]
Presolve removed 12 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.8

Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.300087e+00   0.000000e+00      0s
       5    1.9368987e+00   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.936898717e+00
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0x2cc01101
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 3e+00]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.248924e+00   0.000000e+


Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.414103e+00   0.000000e+00      0s
       6    2.6431358e+00   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.643135804e+00
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0x8266065c
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 3e+00]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.831962e+00   0.000000e+00      0s
       7    1.6216733e+00   0.000000e+00   0.000000e+0

  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 3e+00]
Presolve removed 12 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0857892e-01   3.548709e+00   0.000000e+00      0s
       5    1.1369395e+00   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.136939512e+00
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0x7799273a
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 3e+00]
Presolve removed 16 rows and 9 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 12 nonzeros

Iteration    

  RHS range        [1e-01, 3e+00]
Presolve removed 12 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.2754061e-01   2.856145e+00   0.000000e+00      0s
       3    1.4536775e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.453677529e+00
Set parameter IterationLimit to value 100
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 20 rows, 15 columns and 70 nonzeros
Model fingerprint: 0x7c464bb3
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 3e+00]
Presolve removed 12 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Du

In [ ]:
from scipy.io import loadmat
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import math
from sklearn.model_selection import train_test_split
#Required files:
#*.mat -> containing features(X) and classes(y)
#*fold.mat -> containing the indices of each fold
#*info.mat -> containg an array of the feature size denoting whether a feature is int valued(1) or real valued(0) 
# Define parameters for EigenSample - these can be changed according to the dataset
Cparam = 1
epsilon = 0.01
dataset_list=[2]#[13,16,19,2,21,23,24,26,27,5]
iter_lim = 1e2 #GRB.INFINITY
for dataset in dataset_list:
  Results = list()
  Results_arr = list()
  print('dataset:'+str(dataset))
  annots = loadmat('{}.mat'.format(dataset))
  annotsi = loadmat('{}fold.mat'.format(dataset))
  annotsinf = loadmat('{}info.mat'.format(dataset))
  X = annots['x']
  y = annots['y']
  indices = annotsi['indices']
  indices = indices.reshape(indices.shape[0])
  zi_type = annotsinf['zi_type']

  y = y.reshape(y.shape[0])
  for i in range(y.shape[0]):
    if y[i] == -1:
      y[i] = 0  

  y_fold_min = math.inf
  for i in np.unique(indices):
    y_fold=y[indices!=i]
    y_fold_min = min(y_fold_min,y_fold.shape[0])

  for samples_at_a_time in [y_fold_min-5]:#[y_fold_min-2,y_fold_min-5]
    for eigenvariance in [1,0.98,0.95]:#[1,0.98,0.95,0.9]:
      acc_orig_list = list()
      acc_tot_list = list()
      x_tr_list = list()
      tot_x_list = list() 
      y = y.reshape(-1,1)
      for i in np.unique(indices):
        X_Te = X[indices==i]
        y_Te = y[indices==i]
        X_Tr = X[indices!=i]
        y_Tr = y[indices!=i]

        gen_X,gen_y= generate_samples(X_Tr,y_Tr,len(np.unique(y_Tr.reshape(-1))),Cparam, epsilon, \
                                      samples_at_a_time,zi_type,eigenvariance)
        
        print("gen_X_shape:"+str(gen_X.shape))
        print("gen_y_shape:"+str(gen_y.shape))
        tot_X=np.append(X_Tr,gen_X,axis=0) 
        tot_y=np.append(y_Tr,gen_y,axis=0)
        print("tot_X_shape:"+str(tot_X.shape))
        print("tot_y_shape:"+str(tot_y.shape))
        x_tr_list.append(X_Tr.shape[0])
        tot_x_list.append(tot_X.shape[0])
        try:
          acc_orig,acc_tot=train_test(X_Tr,y_Tr,gen_X,gen_y,tot_X,tot_y,X_Te,y_Te)
          acc_orig_list.append(acc_orig)
          acc_tot_list.append(acc_tot)
        except:
          print('except_occured')
      
      acc_orig_arr = np.array(acc_orig_list)
      acc_tot_arr = np.array(acc_tot_list)
      acc_orig_mean = np.mean(acc_orig_arr)
      acc_tot_mean = np.mean(acc_tot_arr)
      acc_orig_std = np.std(acc_orig_arr)
      acc_tot_std = np.std(acc_tot_arr)
      
      dic = dict()
      li = list()
      dic['dataset'] = dataset
      li.append(dataset)
      dic['parameter'] = samples_at_a_time
      li.append(samples_at_a_time)
      dic['eigenvar'] = eigenvariance
      li.append(eigenvariance)
      dic['orig_shape_0'] = np.mean(np.array(x_tr_list))
      li.append(X_Tr.shape[0])
      dic['orig_shape_1'] = X_Tr.shape[1]
      li.append(X_Tr.shape[1])
      dic['acc_orig_mean'] = acc_orig_mean
      li.append(acc_orig_mean)
      dic['acc_orig_std'] = acc_orig_std
      li.append(acc_orig_std)
      dic['orig_acc_fold'] = str(acc_orig_list).replace(',','/') 
      li.append(str(acc_orig_list).replace(',','/') )
      dic['tot_shape_0'] = np.mean(np.array(tot_x_list))
      li.append(tot_X.shape[0])
      dic['acc_tot_mean'] = acc_tot_mean
      li.append(acc_tot_mean)
      dic['acc_tot_std'] = acc_tot_std
      li.append(acc_tot_std)
      dic['tot_acc_fold'] = str(acc_tot_list).replace(',','/')  
      li.append(str(acc_tot_list).replace(',','/') )
      np.savetxt("eigenbag_dataset_{}_samples_{}_eigenvar_{}.csv".format(dataset,samples_at_a_time,eigenvariance),np.array(li).reshape(1,len(li)),fmt= '%s',delimiter=',',\
             header='dataset, parameter, eigenvar, orig_shape_0, orig_shape_1, acc_orig_mean, acc_orig_std, orig_acc_fold, tot_shape_0, acc_to_mean, acc_tot_std, tot_acc_fold')
      Results.append(dic) 
      Results_arr.append(li)
  print("Results:")
  print(Results)
  Results_arr=np.array(Results_arr)
  np.savetxt("Results_eigenbag_{}.csv".format(dataset),Results_arr,fmt= '%s',delimiter=',',\
             header='dataset, parameter, eigenvar, orig_shape_0, orig_shape_1, acc_orig_mean, acc_orig_std, orig_acc_fold, tot_shape_0, acc_to_mean, acc_tot_std, tot_acc_fold')

dataset:2
outer_counter:0
inner_counter:0
similarity:0.9992488565068331


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


similarity.........:0.9996180164659781
Added
generated_data_size:91
outer_counter:1
inner_counter:1
similarity:0.6403963671930435
outer_counter:2
inner_counter:1
similarity:0.9949149245103982


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


similarity.........:0.9992613054208714
Added
generated_data_size:182
outer_counter:3
inner_counter:2
similarity:0.6078021442536148
outer_counter:4
inner_counter:2
similarity:0.9971658721399583


...

similarity.........:0.7889552847794509
Added
generated_data_size:728
outer_counter:37
inner_counter:29
similarity:0.998626038019572


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


similarity.........:0.7138808915922857
Added
generated_data_size:819
outer_counter:38
inner_counter:30
new in_thres:0.6833082459897637
similarity:0.9952420899335866


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


similarity.........:0.7460727619705648
Added
generated_data_size:910
gen_X_shape:(910, 6)
gen_y_shape:(910, 1)
tot_X_shape:(1006, 6)
tot_y_shape:(1006, 1)
Epoch 1/50
6/6 [==============================] - 1s 40ms/step - loss: 0.6931 - accuracy: 0.5104 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 2/50
6/6 [==============================] - 0s 12ms/step - loss: 0.6920 - accuracy: 0.5104 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/50
6/6 [==============================] - 0s 10ms/step - loss: 0.6910 - accuracy: 0.5104 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 4/50
6/6 [==============================] - 0s 8ms/step - loss: 0.6899 - accuracy: 0.5104 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 5/50
6/6 [==============================] - 0s 10ms/step - loss: 0.6884 - accuracy: 0.5104 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 6/50
6/6 [==============================] - 0s 13ms/step - loss: 0.6863 - accuracy: 0.5104 - val_loss: 0.6915 - val_accuracy: 0.5000
Epoch 7/50
6

In [16]:
len(X_final)

2060

In [17]:
len(Y_final)

2060